<a href="https://colab.research.google.com/github/sword-ace/Using-SGNN-for-Depression-Estimate/blob/main/build_graph_4_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install word2vec

In [ ]:
# !pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.10.0+cu113.html
# !pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.10.0+cu113.html
# !pip install -q torch-geometric 

In [ ]:
# from transformers import BertModel, BertTokenizer, XLNetModel, XLNetTokenizer

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk import PorterStemmer, WordNetLemmatizer
import pandas as pd

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
import re
import string
import torch
stops = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [ ]:

#Text cleaning function
def preprocessingText(text):
  # text = text.lower() #text to lowercase
  # text = re.sub(r'&lt;', '', text) #remove '&lt;' tag
  # text = re.sub(r'<.*?>', '', text) #remove html
  # text = re.sub(r'[0-9]+', '', text) #remove number
  # text = " ".join([word for word in text.split() if word not in stops]) #remove stopwords
  # text = re.sub(r'[^\w\s]', '', text) #remove punctiation
  # text = re.sub(r'[^\x00-\x7f]', '', text) #remove non ASCII strings
  # text = re.sub('\[.*?\]', '', text)
  # text = re.sub(r'http', '', text)
  # text = re.sub(r'www', '', text)
  # text = re.sub(r'https', '', text)
  # text = re.sub('<.*?>+', '', text)
  # text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
  # text = re.sub('\n', '', text)
  # text = re.sub('\w*\d\w*', '', text)
  for c in ['\r', '\n', '\t'] :
    text = re.sub(c, ' ', text) #replace newline and tab with tabs
  # text = re.sub('\s+', ' ', text) #replace multiple spaces with one space
  text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

  return text

In [ ]:
# import torch
import networkx as nx
import torch.nn as nn
from torch import Tensor as Tensor
from torch.nn import Linear as Linear
import torch.nn.init as init
from torch.nn.init import _calculate_correct_fan, calculate_gain
import torch.nn.functional as F
# from torch_geometric.nn import MessagePassing
from torch_geometric import data
import os
import numpy as np

In [ ]:

# ###this is for loading your text file 
# data_train = pd.read_pickle(file_address) 

# text_train = data_train.text

# preprocessed_text_train = [preprocessingText(text) for text in text_train]
# # print(data_train.Labels.values)

# y_train = torch.from_numpy(np.vstack(data_train.Labels.values)).float()

# print('AFTER CLEANING: {}'. format(preprocessed_text_train[0]))

In [ ]:
use_cuda = True
gpu = 0
if use_cuda:
      device = torch.device("cuda:" + str(gpu))
      torch.cuda.set_device(gpu)
      os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu)
else:
    device = "cpu"
device = device
# args.kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
print('Using device:', device)

Using device: cuda:0


This is for preprecessing the raw text 

In [ ]:
def content_gen(preprocessed_text_train):
  all_content=[]
  for doc in preprocessed_text_train:
    words = doc.split()
    all_content.append(words)
  return all_content

##this is prepraed for loading pre-trained embeddings

In [ ]:
embed_size = 200
 
class GloveTokenizer:
    def __init__(self, filename, unk='<unk>', pad='<pad>'):  #'<pad>'
        self.filename = filename
        self.unk = unk
        self.pad = pad
        self.stoi = dict()
        self.itos = dict()
        self.embedding_matrix = [] #list()
        with open(filename, 'r', encoding='utf8') as f: # Read tokenizer file
            for i, line in enumerate(f):
                values = line.split()
                self.stoi[values[0]] = i
                self.itos[i] = values[0]
                self.embedding_matrix.append([float(v) for v in values[1:]])
        if self.unk is not None: # Add unk token into the tokenizer
            i += 1
            self.stoi[self.unk] = i
            self.itos[i] = self.unk
            self.embedding_matrix.append(np.random.rand(embed_size))
        if self.pad is not None: # Add pad token into the tokenizer
            i += 1
            self.stoi[self.pad] = i
            self.itos[i] = self.pad
            self.embedding_matrix.append(np.zeros(embed_size))

        self.embedding_matrix =np.array(self.embedding_matrix) #.astype(np.float32) # Convert if from double to float for efficiency
       

    def encode(self, sentence):
        if type(sentence) == str:
            sentence = sentence.split(' ')
        elif len(sentence): # Convertible to list
            sentence = list(sentence)
        else:
            raise TypeError('sentence should be either a str or a list of str!')
        encoded_sentence = []
        for word in sentence:
            encoded_sentence.append(self.stoi.get(word, self.stoi[self.unk]))
        return encoded_sentence

    def decode(self, encoded_sentence):
        try:
            encoded_sentence = list(encoded_sentence)
        except Exception as e:
            print(e)
            raise TypeError('encoded_sentence should be either a str or a data type that is convertible to list type!')
        sentence = []
        for encoded_word in encoded_sentence:
            sentence.append(self.itos[encoded_word])
        return sentence

    def embedding(self, encoded_sentence):
        return self.embedding_matrix[np.array(encoded_sentence)]


tokenizer = GloveTokenizer(f'/content/drive/MyDrive/glove.6B.{embed_size}d.txt')

In [ ]:
#this is to get total word with freq
def build_vocab(train_dataset, tokenizer):
    stoi = {'<unk>': 0, '<pad>': 1} # Re-index
    itos = {0: '<unk>', 1: '<pad>'} # Re-index
    vocab_count = len(stoi)
    
    vocab_list = [sentence for sentence in train_dataset]
    # print(len(vocab_list))
    
    unique_vocab = []
  
    for vocab in vocab_list:
        unique_vocab.extend(vocab)
    
    unique_vocab = list(set(unique_vocab))
    
    # print(len(unique_vocab))
    # print(unique_vocab)
    for vocab in unique_vocab:
        if vocab in tokenizer.stoi.keys():
            stoi[vocab] = vocab_count
            itos[vocab_count] = vocab
            vocab_count += 1
    #### ---------get the vocab glove embedding ------#####
    # embedding_matrix  = None
    embedding_matrix = tokenizer.embedding(tokenizer.encode(list(stoi.keys())))
    return vocab_count, stoi, embedding_matrix , itos

## This is to generate a random embedding for each word

In [ ]:
rand_embedding_matrix = np.random.uniform(-1,1, (20000,32))

def embed_etract(vocab, embedding_matrix):
    feature_matrix = []
    
    for word_id in vocab:
        feature_matrix.append(embedding_matrix[word_id])  ## can also be replaced with glove embeddings as well    
    return np.array(feature_matrix)

In [ ]:

def build_nodes(train_doc, stoic): 
    nn =[sentence for sentence in train_doc]
    node_sets= []
    node_vob = []
    for vocab in nn:
        # print(vocab)
        node_vob.append(vocab)
        node_sets.append(stoic.get(vocab, 0)) #0 represents the position of the value of dict

    # print("now the train set contains node sets:", node_sets, len(node_sets))
    return node_sets, node_vob  

def build_contents(train_doc, stoic, itos): 
    
    content_sets = []
    node_vob =[]
    nn =[sentence for sentence in train_doc]
    for vocab in nn:
        node_vob.append(itos.get(vocab))
        content_sets.append(stoic.get(vocab, 0)) #0 represents the position of the value of dict
    return content_sets , node_vob

In [ ]:
def graphcon(doc_ids, ngram = 4):
    edges = []
    old_edge_id = []
    local_vocab = np.sort(doc_ids)
    old_to_new = dict(zip(local_vocab, range(len(local_vocab))))

    for index, src_word_old in enumerate(local_vocab):
        src = old_to_new[src_word_old]    
        for i in range(max(0, index - ngram), min(index + ngram+1, len(local_vocab))):
            dst_word_old = local_vocab[i]
            dst = old_to_new[dst_word_old]
            edges.append([src, dst])
     
    edges_n = []
    edges_n.extend(edges)
        
    return edges_n  

In [ ]:
def graph_data(all_content, stoic, itos, embed_matrix,  y_label):
    
    graph_data = []

    for doc_id in range(len(all_content)):
      
        vocab_get= all_content[doc_id]
      
        node_sets, node_vobs = build_nodes(vocab_get, stoic)
        unq_vob = set(node_sets)
        
        content_sets, nodes = build_contents(unq_vob, stoic, itos)

        # print("content vocab in each trans", vocab_get)
        # print("uniq content lenth in each trans", len(set(node_sets)), len(node_sets))
       
        f = embed_etract(unq_vob, embed_matrix)

        e  = graphcon (content_sets, ngram = 4) #8
        
       ##--------------------------------------------------##
        edges1 = [np.array([edge[0], edge[1]]) for edge in e]
        edge_index = torch.tensor(np.array(edges1).T, dtype=torch.long) #.cuda()
      
        print("edge index size", edge_index[0].size())
        print("edge index size2", edge_index[1].size())
        ####-------------------------------###
   
        ft = torch.tensor(f, dtype=torch.float32) #.cuda()
        y  = torch.tensor(y_label[doc_id], dtype= torch.float)
        graph_data.append(data.Data(x=ft, edge_index= edge_index, y=y))
    
    return  graph_data

In [ ]:

content_train = content_gen(preprocessed_text_train)
vocab_count,stoic, glove_embedding_matrix, itos  =  build_vocab(content_train, tokenizer)
graph_data_list = graph_data(content_train, stoic,itos, rand_embedding_matrix, y_train)

